<a href="https://colab.research.google.com/github/PratikhyaManas/TensorRT/blob/master/YOLOv3_TensorRT_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLOv3 Optimization using NVIDIA's TensorRT**

# Mounting of Google Drive

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Switch the current directory to the project folder of Drive

In [0]:
import os
project_path = '/content/drive/My Drive/ML_Datasets/nvidia'
os.chdir(project_path)

# Check the version of CUDA,TensorFlow

In [3]:
# Checking the Version of CUDA & Tensorflow
!nvcc --version
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
TensorFlow 1.x selected.
1.15.2


**Download the TensorRT package. I have uploaded the package to my drive and provided the Sharable link for faster download**

In [0]:
!wget -O nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb https://drive.google.com/open?id=1GLRcQNpdlDX0cN1LXPBB7LzifJuhj4Vf

# Install the required additional TensorRT package

In [4]:
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216/7fa2af80.pub
!apt-get update
!apt-get install -y --no-install-recommends libnvinfer7=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-plugin7=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvparsers7=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvonnxparsers7=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-bin=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-dev=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-plugin-dev=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvparsers-dev=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvonnxparsers-dev=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-samples=7.0.0-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-doc=7.0.0-1+cuda10.0 
!apt-get install tensorrt
!apt-get install uff-converter-tf

Selecting previously unselected package nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216 (1-1) ...
OK
Get:1 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  InRelease
Ign:1 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  InRelease
Get:2 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release [574 B]
Get:2 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release [574 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release.gpg [836 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release.gpg [836 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Check the TensorRT package installation

In [5]:
!dpkg -l | grep TensorRT

ii  graphsurgeon-tf                                              7.0.0-1+cuda10.0                                  amd64        GraphSurgeon for TensorRT package
ii  libnvinfer-bin                                               7.0.0-1+cuda10.0                                  amd64        TensorRT binaries
ii  libnvinfer-dev                                               7.0.0-1+cuda10.0                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-doc                                               7.0.0-1+cuda10.0                                  all          TensorRT documentation
ii  libnvinfer-plugin-dev                                        7.0.0-1+cuda10.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                                           7.0.0-1+cuda10.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer-samples                                        

# Append the YOLOv3 path to the Python path for importing libraries

In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive/ML_Datasets/nvidia/YOLOv3')

# Import the required libraries

In [7]:
import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
from PIL import Image
from YOLOv3 import utils

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Converting the YOLOv3 model to TensorRT model

In [8]:
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def
frozen_graph = read_pb_graph("/content/drive/My Drive/ML_Datasets/nvidia/model/yolov3_gpu_nms.pb")

your_outputs = ["Placeholder:0", "concat_9:0", "mul_9:0"]
# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=your_outputs,
    max_batch_size=1,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP16") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("/content/drive/My Drive/ML_Datasets/nvidia/model/TensorRT_YOLOv3_2.pb", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

# check how many ops of the original frozen model
all_nodes = len([1 for n in frozen_graph.node])
print("numb. of all_nodes in frozen graph:", all_nodes)

# check how many ops that is converted to TensorRT engine
trt_engine_nodes = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("numb. of trt_engine_nodes in TensorRT graph:", trt_engine_nodes)
all_nodes = len([1 for n in trt_graph.node])
print("numb. of all_nodes in TensorRT graph:", all_nodes)

Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
TensorRT model is successfully stored!
numb. of all_nodes in frozen graph: 8016
numb. of trt_engine_nodes in TensorRT graph: 0
numb. of all_nodes in TensorRT graph: 916


As you can see from the result that TensorRT has significantly optimised the nodes of YOLOv3 model by almost 1/9th.

# Setting the Configuration for the Model Inference

In [0]:
# config
SIZE = [416, 416] #input image dimension
# video_path = 0 # if you use camera as input
video_path = "/content/drive/My Drive/ML_Datasets/nvidia/road2.mp4" # path for video input
classes = utils.read_coco_names('/content/drive/My Drive/ML_Datasets/nvidia/YOLOv3/coco.names')
num_classes = len(classes)
GIVEN_ORIGINAL_YOLOv3_MODEL = "/content/drive/My Drive/ML_Datasets/nvidia/model/yolov3_gpu_nms.pb" # to use given original YOLOv3
TENSORRT_YOLOv3_MODEL = "/content/drive/My Drive/ML_Datasets/nvidia/model/TensorRT_YOLOv3_2.pb" # to use the TensorRT optimized model

# Importing the OpenCV libraries

In [0]:
from google.colab.patches import cv2_imshow 
import cv2

# Running the Inference and Validating

In [0]:
# get input-output tensor
input_tensor, output_tensors = \
utils.read_pb_return_tensors(tf.get_default_graph(),
                             TENSORRT_YOLOv3_MODEL,
                             ["Placeholder:0", "concat_9:0", "mul_9:0"])

# perform inference
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as sess:
    vid = cv2.VideoCapture(video_path)
    while True:
        return_value, frame = vid.read()
        if return_value == False:
            print('ret:', return_value)
            vid = cv2.VideoCapture(video_path)
            return_value, frame = vid.read()
        if return_value:
            image = Image.fromarray(frame)
        else:
            raise ValueError("No image!")
            
        img_resized = np.array(image.resize(size=tuple(SIZE)), 
                               dtype=np.float32)
        img_resized = img_resized / 255.
        prev_time = time.time()

        boxes, scores = sess.run(output_tensors, 
                                 feed_dict={input_tensor: 
                                            np.expand_dims(
                                                img_resized, axis=0)})
        boxes, scores, labels = utils.cpu_nms(boxes, 
                                              scores, 
                                              num_classes, 
                                              score_thresh=0.4, 
                                              iou_thresh=0.5)
        image = utils.draw_boxes(image, boxes, scores, labels, 
                                 classes, SIZE, show=False)

        curr_time = time.time()
        exec_time = curr_time - prev_timeq
        result = np.asarray(image)
        info = "time:" + str(round(1000*exec_time, 2)) + " ms, FPS: " + str(round((1000/(1000*exec_time)),1))
        cv2.putText(result, text=info, org=(50, 70), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(255, 0, 0), thickness=2)
        #cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
        cv2_imshow(result)
        if cv2.waitKey(10) & 0xFF == ord('q'): break
        

As running the inference on Cloud takes more time due to the video input provided so the results were not visible here. After running on my local machine my results were as follows



1.   Frames Per Second(FPS) obtained on TENSORRT_YOLOv3_MODEL was around 33 FPS.

2.   Frames Per Second(FPS) obtained on GIVEN_ORIGINAL_YOLOv3_MODEL was around 20 FPS.

The accuracy of TENSORRT_YOLOv3_MODEL was approximately similar to the GIVEN_ORIGINAL_YOLOv3_MODEL.

We can infer that model optimization using TensorRT gives a significant amount of advantage.

